In [ ]:
!pip install transformers
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks')

     |████████████████████████████████| 2.5MB 6.5MB/s 
     |████████████████████████████████| 901kB 32.6MB/s 
     |████████████████████████████████| 3.3MB 39.8MB/s 
Mounted at /content/gdrive


In [ ]:
# import packages
from sklearn import tree
import sklearn.metrics
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import nltk  #The Natural Language Toolkit
import tensorflow as tf
import transformers
from transformers import BertTokenizer, TFBertModel
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM



In [ ]:
# import transformers
#!pip install -q transformers

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
train = pd.read_csv(r"training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv(r"valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv(r"test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
def normalize(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')
  return df
    
def tokenize_function(example, num_train_examples, max_length):
  return tokenizer([x for x in example[:num_train_examples]], 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

def first_stage_model(max_length, num_train_examples, hidden_size = 200, optimizer=tf.keras.optimizers.Adam()):
    
  """
  Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
  """

  input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask}

  bert_out = bert_model(bert_inputs)


  classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(bert_out[0])

  normalized = tf.keras.layers.LayerNormalization(axis=1)(classification_token)
  
  hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(normalized)

  dropout = tf.keras.layers.Dropout(0.2)(hidden)

  classification = tf.keras.layers.Dense(num_train_examples, activation='softmax',name='classification_layer')(dropout)

  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                        outputs=[classification])                              
  
  classification_model.compile(optimizer=optimizer,
                          loss="sparse_categorical_crossentropy", metrics='accuracy')


  return classification_model
  
def second_stage_model():
  return DecisionTreeClassifier(random_state=0)




"""
    Simplify dataframe and normalize scores based on essay set.
"""

max_length = 350
num_train_examples = 500
df = train

df = df[['essay_set','essay','rater1_domain1','rater2_domain1','domain1_score']]
df = normalize(df)


"""
    Create features based on length, average/std of word length, unique words. 
"""

df['length'] = df['essay'].str.split().str.len()
df['avg_word_len'] = df['essay'].str.split().apply(lambda x: np.mean([len(y) for y in x]))
df['std_word_len'] = df['essay'].str.split().apply(lambda x: np.std([len(y) for y in x]))
df['unique_words'] = df['essay'].str.split().apply(lambda x: len(np.unique(x)))


"""
    Define and encode training and validation data.
"""

y = df.normalized_score

X_train, X_val, Y_train, Y_val = train_test_split(df, y, test_size=0.33, random_state=42)

x_train = tokenize_function(X_train.essay, num_train_examples, max_length)
y_train = Y_train[:num_train_examples]
le_train = preprocessing.LabelEncoder()
le_train.fit(y_train)
y_train_enc = le_train.transform(y_train)

x_val = tokenize_function(X_val.essay, num_train_examples, max_length)
y_val = Y_val[:num_train_examples]
le_val = preprocessing.LabelEncoder()
le_val.fit(y_val)
y_val_enc = le_val.transform(y_val)

"""
    Run a neural network with BERT embeddings.
"""
fsm = first_stage_model(max_length, num_train_examples)
fsm.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask], y_train_enc, 
        validation_data = ([x_val.input_ids, x_val.token_type_ids, x_val.attention_mask], y_val_enc), epochs=5, batch_size=1)
X_fsm = fsm.predict([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask], 
                          batch_size=3, 
                          steps=2)   

"""
    Combine output of first-stage model with other features for second stage model.
"""

X_ext = pd.DataFrame(X_fsm.transpose(), columns=["nn_feature" + str(i) for i in range(X_fsm.shape[0])])
X_train = X_train.reset_index()
X_ext['length'] = X_train['std_word_len'][:num_train_examples]
X_ext['avg_word_len'] = X_train['std_word_len'][:num_train_examples]
X_ext['std_word_len'] = X_train['std_word_len'][:num_train_examples]
X_ext['std_word_len'] = X_train['std_word_len'][:num_train_examples]
X_ext['unique_words'] = X_train['unique_words'][:num_train_examples]

"""
    Concatenate extracted features with engineered features.
"""


X_fsm_val = fsm.predict([x_val.input_ids, x_val.token_type_ids, x_val.attention_mask], 
                          batch_size=3, 
                          steps=2)   
X_ext_val = pd.DataFrame(X_fsm_val.transpose(), columns=["nn_feature" + str(i) for i in range(X_fsm_val.shape[0])])
X_val = X_val.reset_index()

X_ext_val['length'] = X_val['std_word_len'][:num_train_examples]
X_ext_val['avg_word_len'] = X_val['std_word_len'][:num_train_examples]
X_ext_val['std_word_len'] = X_val['std_word_len'][:num_train_examples]
X_ext_val['std_word_len'] = X_val['std_word_len'][:num_train_examples]
X_ext_val['unique_words'] = X_val['unique_words'][:num_train_examples]

"""
    Run second stage model and evaluate/predict.
"""

ssm = second_stage_model()
ssm.fit(X_ext, y_train_enc) 
y_pred_enc = ssm.predict(X_ext)
y_pred_enc_val = ssm.predict(X_ext_val)

print("Training Accuracy: " + str(accuracy_score(y_val_enc, y_pred_enc_val))[:5] + " " + "Validation Accuracy: " + str(accuracy_score(y_train_enc, y_pred_enc)))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

Epoch 1/5
500/500 [==============================] - 3104s 6s/step - loss: 4.1200 - accuracy: 0.1840 - val_loss: 4.9176 - val_accuracy: 0.2240
Epoch 2/5
 74/500 [===>..........................] - ETA: 34:48 - loss: 2.9724 - accuracy: 0.2568

In [ ]:
print("Training Accuracy: " + str(accuracy_score(y_val_enc, y_pred_enc_val))[:5] + " " + "Validation Accuracy: " + str(accuracy_score(y_train_enc, y_pred_enc)))


Training Accuracy: 0.166 Validation Accuracy: 1.0
